# LEA School District Finance Survey – National Level Analysis

#### Note: This project is still in progress

Quickly navigate to the Visualizations & Summaries section by clicking [here](#visualizations_summaries) to bypass the detailed data manipulation code and directly access the consolidated analysis and visual representations.

In [1]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utilities import Utilities

In [2]:
# Create Instance of Utilities (A class with various data operations that will be used throughout the project)
utils = Utilities()

# Establish Database Connection
# The upcoming data retrieval method will switch to CSV format if the database connection fails
conn = utils.create_database_conn()

#### Note on Database Connectivity and Data Analysis

This notebook facilitates a fixed analysis of static data covering the years 2010 to 2020, designed to ensure consistency and replicability of results across different environments. The choice between data extraction methods is streamlined for efficiency and convenience:

- **Direct Database Connection**: The notebook attempts to connect to the database at the outset using provided credentials. A successful connection is preferred, especially for conducting the analysis, as it allows direct access to the database. This enables flexibility in querying additional data or making structural adjustments without immediate concern for updating CSV files. The data in the database is static, mirroring the fixed analysis period, and matches exactly the data exported into the CSV files.

- **CSV File Access as Fallback**: In cases where the database connection cannot be established (due to incorrect credentials, network access issues, or database unavailability), the notebook automatically switches to using pre-exported CSV files (`student_counts_by_state.csv` and `tech_vocational_expenditures_2010_to_2020.csv`). These files are generated using the same queries as those within the notebook, ensuring that the data and subsequent analysis results will match those obtained directly from the database. This method guarantees that the analysis can be consistently replicated, regardless of the environment setup.

The transition between the database and CSV files is handled automatically, eliminating the need for manual configuration. This approach ensures that, whether accessing the database directly or using CSV files, the analysis remains consistent and true to the static dataset from 2010 to 2020.

## Expenditure Analysis

In [3]:
# Query expenditure data
query = """
SELECT 
    state,
    region,
    expenditure_title, 
    year, 
    SUM(amount) AS amount, 
    AVG(amount_z_score) AS amount_z_score_avg 
FROM expenses.expenditure_zscores_by_state_year
WHERE expenditure_title ILIKE '%tech%' 
OR expenditure_title ILIKE '%vocation%'
GROUP BY state, region, year, expenditure_title
ORDER BY state, year, expenditure_title;
"""

In [4]:
if conn:
    tech_vocational_expenditures = utils.execute_sql(query, conn)
else:
        try:
            tech_vocational_expenditures = pd.read_csv(os.path.join('..','tech_vocational_expenditures_2010_to_2020.csv'), 
                                                       dtype={'amount_z_score_avg': 'str'})
        except FileNotFoundError:
            print("CSV file 'tech_vocational_expenditures_2010_to_2020.csv' not found. Please ensure the file is in the correct directory.")
            raise 
        except Exception as e:
            print(f"An error occurred while loading the CSV file: {e}")
            raise 
tech_vocational_expenditures.head()

,state,region,expenditure_title,year,amount,amount_z_score_avg
0,Alabama,South,teacher_salaries_vocational_education,2010.0,110309000.0,-1.67803030303030303E-19
1,Alabama,South,teacher_salaries_vocational_education,2011.0,107614000.0,-6.14166666666666667E-19
2,Alabama,South,teacher_salaries_vocational_education,2012.0,105162000.0,-2.25606060606060606E-19
3,Alabama,South,teacher_salaries_vocational_education,2013.0,108247000.0,1.71343283582089552E-19
4,Alabama,South,teacher_salaries_vocational_education,2014.0,110787000.0,6.56666666666666667E-19


In [5]:
tech_vocational_expenditures.dtypes

state                  object
region                 object
expenditure_title      object
year                  float64
amount                 object
amount_z_score_avg     object
dtype: object

In [6]:
tech_vocational_expenditures = tech_vocational_expenditures.astype({'year' : int,
                                                                    'amount' : 'float64',
                                                                    'amount_z_score_avg' : 'float64'
                                                                    })

tech_vocational_expenditures.dtypes

state                  object
region                 object
expenditure_title      object
year                    int64
amount                float64
amount_z_score_avg    float64
dtype: object

In [7]:
tech_vocational_expenditures.head()

,state,region,expenditure_title,year,amount,amount_z_score_avg
0,Alabama,South,teacher_salaries_vocational_education,2010,110309000.0,-1.678030e-19
1,Alabama,South,teacher_salaries_vocational_education,2011,107614000.0,-6.141667e-19
2,Alabama,South,teacher_salaries_vocational_education,2012,105162000.0,-2.256061e-19
3,Alabama,South,teacher_salaries_vocational_education,2013,108247000.0,1.713433e-19
4,Alabama,South,teacher_salaries_vocational_education,2014,110787000.0,6.566667e-19


In [8]:
query = """
SELECT 
	e.state,
    e.region, 
	DATE_PART('year', stats.year) AS year, 
	SUM(stats.fall_membership) AS student_count
FROM entity.annual_stats AS stats
INNER JOIN entity.entity as e
	ON stats.census_id = e.census_id
GROUP BY e.state, e.region, stats.year
ORDER BY e.state, stats.year;
"""

In [9]:
if conn:
    student_counts_by_state_year = utils.execute_sql(query, conn)
else:
    try:
        student_counts_by_state_year = pd.read_csv(os.path.join('..','student_counts_by_state.csv'))
    except FileNotFoundError:
        print("CSV file 'student_counts_by_state.csv' not found. Please ensure the file is in the correct directory.")
        raise  # Reraise the exception to stop the notebook
    except Exception as e:
        print(f"An error occurred while loading the CSV file: {e}")
        raise  # Reraise the exception to stop the notebook

# Display the head of the DataFrame
student_counts_by_state_year.head()


,state,region,year,student_count
0,Alabama,South,2010.0,748889.0
1,Alabama,South,2011.0,738322.0
2,Alabama,South,2012.0,744621.0
3,Alabama,South,2013.0,744548.0
4,Alabama,South,2014.0,740081.0


In [10]:
student_counts_by_state_year = student_counts_by_state_year.astype({'year' : int,
                                                                    'student_count' : int
                                                                    })
student_counts_by_state_year.dtypes

state            object
region           object
year              int64
student_count     int64
dtype: object

In [11]:
statewise_data_w_student_counts = pd.merge(student_counts_by_state_year, tech_vocational_expenditures, how= 'inner', on= ['state', 'year'], validate="many_to_many")
statewise_data_w_student_counts.head()

,state,region_x,year,student_count,region_y,expenditure_title,amount,amount_z_score_avg
0,Alabama,South,2010,748889,South,teacher_salaries_vocational_education,110309000.0,-1.678030e-19
1,Alabama,South,2011,738322,South,teacher_salaries_vocational_education,107614000.0,-6.141667e-19
2,Alabama,South,2012,744621,South,teacher_salaries_vocational_education,105162000.0,-2.256061e-19
3,Alabama,South,2013,744548,South,teacher_salaries_vocational_education,108247000.0,1.713433e-19
4,Alabama,South,2014,740081,South,teacher_salaries_vocational_education,110787000.0,6.566667e-19


In [12]:
statewise_data_w_student_counts['cost_per_student'] = statewise_data_w_student_counts['amount'] / statewise_data_w_student_counts['student_count']
statewise_data_w_student_counts.head()

,state,region_x,year,student_count,region_y,expenditure_title,amount,amount_z_score_avg,cost_per_student
0,Alabama,South,2010,748889,South,teacher_salaries_vocational_education,110309000.0,-1.678030e-19,147.296862
1,Alabama,South,2011,738322,South,teacher_salaries_vocational_education,107614000.0,-6.141667e-19,145.754833
2,Alabama,South,2012,744621,South,teacher_salaries_vocational_education,105162000.0,-2.256061e-19,141.228894
3,Alabama,South,2013,744548,South,teacher_salaries_vocational_education,108247000.0,1.713433e-19,145.386194
4,Alabama,South,2014,740081,South,teacher_salaries_vocational_education,110787000.0,6.566667e-19,149.695777


### Understanding "Cost Per Student" Calculation

**Methodology:**

For the purpose of this analysis, the "cost per student" metric is calculated as the total expenditure amount divided by the number of students for the given year. The formula used is as follows:

```python
statewise_data_w_student_counts['cost_per_student'] = statewise_data_w_student_counts['amount'] / statewise_data_w_student_counts['student_count']
```

This calculation aims to provide an approximate value indicating the expenditure per student based on the available data of total expenditure and student counts.

**Limitations:**

It's essential to acknowledge the limitations of this calculation. The derived "cost per student" metric does not account for the nuanced breakdown of expenses directly associated with each student. The analysis lacks the granularity to determine what percentage of funds is genuinely allocated towards the student's educational expenses versus other operational or administrative costs. As a result, the calculated "cost per student" serves as an overall indicator and may not accurately represent the true cost of education per student.

In [13]:
total_expenditures_yearly = statewise_data_w_student_counts.groupby('year')['amount'].sum().reset_index()
total_expenditures_yearly.head()

,year,amount
0,2010,4.269554e+09
1,2011,4.359258e+09
2,2012,4.899872e+09
3,2013,4.958659e+09
4,2014,5.264175e+09


In [14]:
yearly_expense_totals_by_title = statewise_data_w_student_counts.groupby(['year', 'expenditure_title'])['amount'].sum().reset_index()
yearly_expense_totals_by_title.head()

,year,expenditure_title,amount
0,2010,teacher_salaries_vocational_education,4.269554e+09
1,2011,teacher_salaries_vocational_education,4.359258e+09
2,2012,teacher_salaries_vocational_education,4.899872e+09
3,2013,teacher_salaries_vocational_education,4.958659e+09
4,2014,teacher_salaries_vocational_education,5.264175e+09


In [15]:
tech_voc_expenditures_line_chart1 = px.line(statewise_data_w_student_counts.groupby('year')['amount'].sum().reset_index(),
                                           x= 'year',
                                           y= 'amount',
                                           title= 'Tech and Vocational Education Expenditures Over Time')

In [16]:
TOTAL_2014 = utils.get_year_total(total_expenditures_yearly, 2014, 'amount')
TOTAL_2016 = utils.get_year_total(total_expenditures_yearly, 2016, 'amount')

TOTAL_DIFFERENCE_2014_TO_2016 = utils.calculate_total_difference(TOTAL_2014, TOTAL_2016)
PERCENT_DIFFERENCE_2014_TO_2016 = utils.calculate_percentage_difference(TOTAL_2014, TOTAL_2016)

# Print out the results
print(f'2014 Amount: {TOTAL_2014}')
print(f'2016 Amount: {TOTAL_2016}')
print(f"Total amount difference between 2014 and 2016: {TOTAL_DIFFERENCE_2014_TO_2016}")
print(f"Percentage difference between 2014 and 2016: {PERCENT_DIFFERENCE_2014_TO_2016:.2f}%")

2014 Amount: 5264175000.0
2016 Amount: 9614528000.0
Total amount difference between 2014 and 2016: 4350353000.0
Percentage difference between 2014 and 2016: 82.64%


In [17]:
tech_voc_expenditures_line_chart2 = px.line(yearly_expense_totals_by_title,
                                           x= 'year',
                                           y= 'amount', 
                                           color='expenditure_title', 
                                           title= 'Expenditures by Title Over Time')

In [18]:
TECH_RELATED_SUPPLIES_SERVICES_2014_TO_2016_TOTAL = yearly_expense_totals_by_title[
    (yearly_expense_totals_by_title['expenditure_title'] == 'tech_related_supplies_services') & 
    (yearly_expense_totals_by_title['year'].isin([2014, 2015, 2016]))
]['amount'].sum()

print(f'Tech Related Supplies & Services Total: {TECH_RELATED_SUPPLIES_SERVICES_2014_TO_2016_TOTAL}')

Tech Related Supplies & Services Total: 3948099000.0


In [19]:
TECH_RELATED_EQUIPMENT_2014_TO_2016_TOTAL = yearly_expense_totals_by_title[
    (yearly_expense_totals_by_title['expenditure_title'] == 'tech_related_equipment') & 
    (yearly_expense_totals_by_title['year'].isin([2014, 2015, 2016]))
]['amount'].sum()

print(f'Tech Related Equipment Total: {TECH_RELATED_EQUIPMENT_2014_TO_2016_TOTAL}')

Tech Related Equipment Total: 1263890000.0


In [20]:
TEACHER_SALARIES_VOCATIONAL_ED_2014_TO_2016_TOTAL = yearly_expense_totals_by_title[
    (yearly_expense_totals_by_title['expenditure_title'] == 'teacher_salaries_vocational_education') & 
    (yearly_expense_totals_by_title['year'].isin([2014, 2015, 2016]))
]['amount'].sum()

print(f'Teacher Salaries Vocational Education: {TEACHER_SALARIES_VOCATIONAL_ED_2014_TO_2016_TOTAL}')

Teacher Salaries Vocational Education: 16308173000.0


In [21]:
GRAND_TOTAL_2014_TO_2016 = TECH_RELATED_SUPPLIES_SERVICES_2014_TO_2016_TOTAL + TECH_RELATED_EQUIPMENT_2014_TO_2016_TOTAL + TEACHER_SALARIES_VOCATIONAL_ED_2014_TO_2016_TOTAL

print(f'2014 to 2016 Grand Total: ${GRAND_TOTAL_2014_TO_2016}')

2014 to 2016 Grand Total: $21520162000.0


In [22]:
PERCENT_DIFFERENCE_BETWEEN_2014_2016 = TOTAL_DIFFERENCE_2014_TO_2016 / TOTAL_2014 * 100

TECHER_SALARIES_VOCATIONAL_ED_2014_TO_2016_PERCENT = TEACHER_SALARIES_VOCATIONAL_ED_2014_TO_2016_TOTAL / GRAND_TOTAL_2014_TO_2016 * 100

TECH_RELATED_SUPPLIES_SERVICES_2014_TO_2016_PERCENT = TECH_RELATED_SUPPLIES_SERVICES_2014_TO_2016_TOTAL / GRAND_TOTAL_2014_TO_2016 * 100

TECH_RELATED_EQUIPMENT_2014_TO_2016_PERCENT = TECH_RELATED_EQUIPMENT_2014_TO_2016_TOTAL / GRAND_TOTAL_2014_TO_2016 * 100

print(f'Difference between 2014 and 2016: ${TOTAL_DIFFERENCE_2014_TO_2016}')
print(f'Percent difference between 2014 and 2016: {PERCENT_DIFFERENCE_BETWEEN_2014_2016:.2f}%')
print(f'Teacher Salaries Vocational Education % of Total Revenue between 2014 and 2016: {TECHER_SALARIES_VOCATIONAL_ED_2014_TO_2016_PERCENT:.2f}%')
print(f'Tech Related Supplies & Services % of Total Revenue between 2014 and 2016: {TECH_RELATED_SUPPLIES_SERVICES_2014_TO_2016_PERCENT:.2f}%')
print(f'Tech Related Equipment % of Total Revenue between 2014 and 2016: {TECH_RELATED_EQUIPMENT_2014_TO_2016_PERCENT:.2f}%')

Difference between 2014 and 2016: $4350353000.0
Percent difference between 2014 and 2016: 82.64%
Teacher Salaries Vocational Education % of Total Revenue between 2014 and 2016: 75.78%
Tech Related Supplies & Services % of Total Revenue between 2014 and 2016: 18.35%
Tech Related Equipment % of Total Revenue between 2014 and 2016: 5.87%


In [23]:
TOTAL_2015 = utils.get_year_total(total_expenditures_yearly, 2015, 'amount')

In [24]:
tech_vocational_line_charts_combined = utils.create_combined_figure(tech_voc_expenditures_line_chart1, 
                                            tech_voc_expenditures_line_chart2, 
                                            'Tech and Vocational Education Expenditures Comparison',
                                            ('Total Expenditures Over Time', 'Expenditures by Title Over Time')
                                            )

tech_vocational_line_charts_combined.update_layout(
    annotations=[
        dict(
            x=2015,
            y=TOTAL_2015 - 100000000,
            xref="x",
            yref="y",
            text="NCLB End, ESSA Start",
            showarrow=True,
            arrowhead=5,
            ax=60,
            ay=50
        )
    ]
)

tech_vocational_line_charts_combined.show()

### Notes:

Original Analysis:  

**Total Expenditures Over Time:**  
- There is a steady positive growth in Tech and Vocational expenditures from 2010 to 2014.
- In 2014, there is a sharp increase in expenditures ending in 2016.
- In 2016, expenditures appear to steady.
- Positive growth is visible throughout. 

**Expenditures by Title Over Time:**  
- After breaking the data out by Expenditure Title, we see that in 2015, two additional expenditure categories were added:
    - Tech Related Supplies & Services
    - Tech Related Equipment
- There is fairly consistent positive growth in Tech Related Equipment between 2015 and 2019.
- After 2019, there is a slight decline in this expenditure category. 
    - It should be noted that COVID-19 started spreading in late 2019, which lead to worldwide pandemic in 2020. 
- Tech Related Supplies & Services appears to have the largest impact to the growth exhibited between 2015 and 2020.
- Teacher Salaries Vocational Education remains constant.  

### Additional Analysis:  
- Difference between 2014 & 2016: $ 4,350,353,000  
- Percent difference between 2014 & 2016: 82.64%  

- Teacher Salaries Vocational Education % of Total Tech and Vocational Expenditures from 2014 to 2016: 75.78%  
    - Total: $16,308,173,000  

- Tech Related Supplies & Services percent of Total Tech and Vocational Expenditures from 2014 to 2016: 18.35%  
    - Total: $3,948,099,000  
    
- Tech Related Equipment percent of Total Tech and Vocational Expenditures from 2014 to 2016: 5.87%  
    - Total: $1,263,890,000

#### Summary:  
Between 2014 and 2016, there was an 82.64% increase in tech and vocational education related expenditures. Upon further investigation, this increase was driven by the addition of two expenditure categories, `Tech Related Supplies Services` and `Tech Related Equipment`. The main driving factor in the expenditure spike was due to `Tech Related Supplies Services`, which accounted for 18.35% of the total expenditures between 2014 and 2016. Although we see consistent growth in `Teacher Salaries Vocational Edication`, this still accounted for 75.78% of expenditures between 2014 and 2016. After 2016, the sharp spike in expenditures slows, but still maintains a positive trend. `Tech Related Supplies Services` continues to have a greater influence to the overall trend despite the spike in expenditures slowing after 2016.

#### Noteworthy Event in 2015:

The **Every Student Succeeds Act (ESSA)**, enacted in 2015, marked the transition from the **No Child Left Behind Act (NCLB)** and likely impacted educational spending trends. Both **NCLB** and **ESSA** were centered on standards-based education reform, emphasizing accountability and federal funding distribution across various educational domains. However, their primary focus wasn't explicitly geared toward technology and vocational education allocations. Despite lacking specific dedicated funding, their broader influence on education funding, accountability measures, and state discretion might have indirectly affected these areas.

#### Food for Thought: 
- There is a steady increase in `Teacher Salaries Vocational Education` over the last decade. Is this change driven by higher teacher salaries, more vocational class offerings resulting in more teachers being hired, or is it a combination of both?
- What are the rankings of expenditures by state?
- What are the rankings of individual school districts within each state that are driving these trends?
- Do student counts per school district/state have any impact on these trends?

In [25]:
yearly_trends_w_cost_per_student = statewise_data_w_student_counts.groupby(['year', 'expenditure_title'])['cost_per_student'].sum().reset_index()
yearly_trends_w_cost_per_student.head()

,year,expenditure_title,cost_per_student
0,2010,teacher_salaries_vocational_education,4967.648605
1,2011,teacher_salaries_vocational_education,5143.232523
2,2012,teacher_salaries_vocational_education,5135.916404
3,2013,teacher_salaries_vocational_education,5184.779075
4,2014,teacher_salaries_vocational_education,5512.360265


In [26]:
annual_total_cost_per_student = yearly_trends_w_cost_per_student.groupby('year')['cost_per_student'].sum().reset_index()
COST_PER_STUDENT_2014 = annual_total_cost_per_student[annual_total_cost_per_student['year'] == 2014]['cost_per_student'].sum()
COST_PER_STUDENT_2015 = annual_total_cost_per_student[annual_total_cost_per_student['year'] == 2015]['cost_per_student'].sum()
COST_PER_STUDENT_2016 = annual_total_cost_per_student[annual_total_cost_per_student['year'] == 2016]['cost_per_student'].sum()

print(f'2014 Total: ${COST_PER_STUDENT_2014:.2f}')
print(f'2015 Total: ${COST_PER_STUDENT_2015:.2f}')
print(f'2016 Total: ${COST_PER_STUDENT_2016:.2f}')

2014 Total: $5512.36
2015 Total: $8316.33
2016 Total: $11817.46


In [27]:
COST_PER_STUDENT_DIFFERENCE_BETWEEN_2014_2016 = COST_PER_STUDENT_2016 - COST_PER_STUDENT_2014

COST_PER_STUDENT_PERCENT_DIFFERENCE_BETWEEN_2014_2016 = utils.calculate_percentage_difference(COST_PER_STUDENT_2014, COST_PER_STUDENT_2016)

print(f'Difference between 2014 and 2016: ${COST_PER_STUDENT_DIFFERENCE_BETWEEN_2014_2016:.2f}')
print(f'Percent difference between 2014 and 2016: {COST_PER_STUDENT_PERCENT_DIFFERENCE_BETWEEN_2014_2016:.2f}%')

Difference between 2014 and 2016: $6305.10
Percent difference between 2014 and 2016: 114.38%


In [28]:
yearly_trends_w_cost_per_student = yearly_trends_w_cost_per_student.sort_values(by=['expenditure_title', 'year'])

# Calculate the year-to-year growth rate for each category
yearly_trends_w_cost_per_student['growth_rate'] = yearly_trends_w_cost_per_student.groupby('expenditure_title')['cost_per_student'].pct_change()
yearly_trends_w_cost_per_student['growth_rate'] *= 100

yearly_trends_w_cost_per_student.head()

,year,expenditure_title,cost_per_student,growth_rate
20,2020,cares_act_expenditure_tech_related_equipment,69.398844,NaN
21,2020,cares_act_expenditure_tech_related_supplies_se...,175.002068,NaN
0,2010,teacher_salaries_vocational_education,4967.648605,NaN
1,2011,teacher_salaries_vocational_education,5143.232523,3.534548
2,2012,teacher_salaries_vocational_education,5135.916404,-0.142247


In [29]:
yearly_trends_w_cost_per_student.dtypes

year                   int64
expenditure_title     object
cost_per_student     float64
growth_rate          float64
dtype: object

In [31]:
tech_vocational_salaries_cost_per_student_growth_rate_2013_2019 = utils.calculate_mean_growth_rate(yearly_trends_w_cost_per_student,
                                                                                                   2013, 2019,
                                                                                                   expenditure_title= 'teacher_salaries_vocational_education'
                                                                                                   )

TECH_RELATED_EQUIPMENT_COST_PER_STUDENT_GROWTH_RATE_2015_2016 = utils.calculate_mean_growth_rate(yearly_trends_w_cost_per_student,
                                                                                                 2015, 2016,
                                                                                                 expenditure_title= 'tech_related_equipment'
                                                                                                 )

TECH_RELATED_EQUIPMENT_COST_PER_STUDENT_GROWTH_RATE_2016_2020 = utils.calculate_mean_growth_rate(yearly_trends_w_cost_per_student,
                                                                                                 2016, 2020,
                                                                                                 expenditure_title= 'tech_related_equipment'
                                                                                                 )

TECH_RELATED_SUPPLIES_SERVICES_COST_PER_STUDENT_GROWTH_RATE_2015_2016 = utils.calculate_mean_growth_rate(yearly_trends_w_cost_per_student,
                                                                                                         2015, 2016,
                                                                                                         expenditure_title= 'tech_related_supplies_services'
                                                                                                         )

TECH_RELATED_SUPPLIES_SERVICES_COST_PER_STUDENT_GROWTH_RATE_2016_2020 = utils.calculate_mean_growth_rate(yearly_trends_w_cost_per_student,
                                                                                                         2016, 2020,
                                                                                                         expenditure_title= 'tech_related_supplies_services'
                                                                                                         )

print(f"Average cost per student growth rate of Teacher Salaries - Vocational Education between 2013 and 2019: {tech_vocational_salaries_cost_per_student_growth_rate_2013_2019:.2f}%")
print(f"Average cost per student growth rate of Tech Related Equipment between 2015 and 2016: {TECH_RELATED_EQUIPMENT_COST_PER_STUDENT_GROWTH_RATE_2015_2016:.2f}%")
print(f"Average cost per student growth rate of Tech Related Equipment between 2016 and 2020: {TECH_RELATED_EQUIPMENT_COST_PER_STUDENT_GROWTH_RATE_2016_2020:.2f}%")
print(f"Average cost per student growth rate of Tech Related Supplies & Services between 2015 and 2016: {TECH_RELATED_SUPPLIES_SERVICES_COST_PER_STUDENT_GROWTH_RATE_2015_2016:.2f}%")
print(f"Average cost per student growth rate of Tech Related Supplies & Services between 2016 and 2020: {TECH_RELATED_SUPPLIES_SERVICES_COST_PER_STUDENT_GROWTH_RATE_2016_2020:.2f}%")

Average cost per student growth rate of Teacher Salaries - Vocational Education between 2013 and 2019: 3.27%
Average cost per student growth rate of Tech Related Equipment between 2015 and 2016: 68.56%
Average cost per student growth rate of Tech Related Equipment between 2016 and 2020: 21.36%
Average cost per student growth rate of Tech Related Supplies & Services between 2015 and 2016: 146.71%
Average cost per student growth rate of Tech Related Supplies & Services between 2016 and 2020: 42.20%


In [ ]:
# Create Line Chart for Cost per Student Expenses
tech_voc_cost_per_student_line_chart = px.line(annual_total_cost_per_student,
                                               x= 'year',
                                               y= 'cost_per_student',
                                               title= 'Tech and Vocational Cost per Student by Year')

In [ ]:
# Create Line Chart for Cost per Student Expenses by Expenditure Title
tech_voc_cost_per_student_line_chart1 = px.line(yearly_trends_w_cost_per_student,
                                               x= 'year',
                                               y= 'cost_per_student', 
                                               color='expenditure_title', 
                                               title= 'Tech and Vocational Cost per Student by Year & Expenditure Title')


In [ ]:
# Create Line Chart for Cost per Student Expenses by Expenditure Title
tech_voc_cost_per_student_line_chart2 = px.line(yearly_trends_w_cost_per_student,
                                               x= 'year',
                                               y= 'growth_rate', 
                                               color='expenditure_title', 
                                               title= 'Tech and Vocational Cost per Student by Year & Expenditure Title')

tech_voc_cost_per_student_line_chart2.show()

In [ ]:
tech_vocational_cost_per_student_line_charts_combined = utils.create_combined_figure(
    tech_voc_cost_per_student_line_chart,
    tech_voc_cost_per_student_line_chart1,
    'Tech and Vocational Education Costs per Student Comparison',
    ('Tech & Vocational Cost per Student Over Time', 'Tech & Vocational Cost per Student by Title Over Time')
)

tech_vocational_cost_per_student_line_charts_combined.show()

### Notes:
The cost per student in tech and vocational education expenditures reveals significant growth trends, especially between 2014 and 2016, as observed in the line charts. This period marked a substantial 114.38% increase in costs per student, totaling an increase of $6305.10.

**Teacher Salaries - Vocational Education**
- `teacher_salaries_vocational_education` demonstrates an overall upward trend.
- The average annual growth rate from 2013 to 2019 was 3.27%.
- A significant rise of 7.35% was recorded in 2020, indicating an exceptional growth year.

**Tech-Related Equipment**
- The tech_related_equipment category experienced its highest growth between 2015 and 2016, at 68.56%.
- From 2016 to 2020, the average annual growth rate was 21.36%, showing sustained growth over the period.

**Tech-Related Supplies & Services**
- The tech_related_supplies_services category witnessed its most substantial increase between 2015 and 2016, at 146.71%.
- Although the growth rate decreased after 2016, it still maintained a high average annual growth rate of 42.20% until 2020.

In [ ]:
yearly_trends_w_cost_per_student.head(25)

## Student Population Analysis

In [ ]:
student_counts_by_year = student_counts_by_state_year[['year', 'student_count']].groupby('year')['student_count'].sum().reset_index()
student_counts_by_year.head(11)

In [ ]:
student_population_bar = px.bar(student_counts_by_year,
                                x= 'year',
                                y= 'student_count',
                                log_y= True,
                                title= 'Student Population by Year')

In [ ]:
student_counts_by_year['difference_from_prev_year'] = student_counts_by_year['student_count'].diff()
student_counts_by_year.head(11)

In [ ]:
# Create the waterfall chart
student_counts_waterfall = go.Figure(go.Waterfall(
    name="Year to Year Changes in Student Population",
    orientation="v",
    measure=["relative"] * len(student_counts_by_year),  # Use relative measure for each value
    x=student_counts_by_year['year'],
    textposition="outside",
    y=student_counts_by_year['difference_from_prev_year']
))

In [ ]:
STUDENT_COUNT_2010 = utils.get_year_total(student_counts_by_year, 2010, 'student_count')
STUDENT_COUNT_2015 = utils.get_year_total(student_counts_by_year, 2015, 'student_count')
STUDENT_COUNT_TOTAL_DIFF_2010_2015 = utils.calculate_total_difference(STUDENT_COUNT_2010, STUDENT_COUNT_2015)
STUDENT_COUNT_PERCENT_DIFF_2010_2015 = utils.calculate_percentage_difference(STUDENT_COUNT_2010, STUDENT_COUNT_2015)

print(f"Student Population in 2010: {STUDENT_COUNT_2010}")
print(f"Student Population in 2015: {STUDENT_COUNT_2015}")
print(f"Difference between 2010 and 2015: {STUDENT_COUNT_TOTAL_DIFF_2010_2015}")
print(f"Percent Difference between 2010 and 2015: {STUDENT_COUNT_PERCENT_DIFF_2010_2015:.2f}%")

In [ ]:
student_counts_waterfall.update_layout(
    title='Student Population Change from Previous Year',
    xaxis_title='Year',
    yaxis_title='Population Change'
)

student_population_combined = utils.create_combined_figure(student_population_bar, 
                                                           student_counts_waterfall,
                                                           "Student Population Trends Over Time",
                                                           ('Student Population by Year','Student Population Change from Previous Year'))

student_population_combined.update_yaxes(type="log", row=1, col=1)

student_population_combined.show()

#### Notes:
- The student population between 2010 and 2015 grew by approximately 215,966 individuals, resulting in a percentage increase of about 0.45%. Despite the decline observed in 2012, the overall trend still showcases substantial growth in the student population over this five-year span.

## Funding Analysis

### Analysis Scope and Limitations

**Funding Titles Included:**

The analysis encompasses specific funding titles across federal and state sources. These titles were chosen based on their potential contribution to expenditure trends related to tech/vocational education. The included funding titles are as follows:

#### Federal Funding Sources:
- "21st_century_learning_centers_thru_state"
- "bilingual_education_thru_state"
- "effective_instruction_support_thru_state"
- "esser_fund"
- "geer_fund"
- "indiv_with_disabilities_thru_state"
- "math_science_teacher_quality_thru_state"
- "student_support_academic_enrich_thru_state"
- "title_I_thru_state"
- "voc_tech_education_thru_state"

#### State Funding Sources:
- "bilingual_education_state"
- "compensatory_basic_skills_programs"
- "general_formula_assistance"
- "special_education_programs"
- "vocational_education_programs"

**Analysis Overview:**

The subsequent analysis presents a generalized view of expenditure trends attributed to these specific funding categories. However, it's important to note that while these categories might provide insights into the overall spending patterns, the analysis lacks granularity. We cannot determine the exact amount allocated or spent specifically towards tech/vocational education within each category. Therefore, the trends identified serve as indicators or drivers rather than precise figures for tech/vocational education expenditure.

In [ ]:
query = "SELECT * FROM revenue.funding_view;"

In [ ]:
if conn:
    funding_data = utils.execute_sql(query, conn)
else:
    try:
        funding_data = pd.read_csv(os.path.join('..','funding_data.csv'))
    except FileNotFoundError:
        print("CSV file 'fudning_data.csv' not found. Please ensure the file is in the correct directory.")
        raise  # Reraise the exception to stop the notebook
    except Exception as e:
        print(f"An error occurred while loading the CSV file: {e}")
        raise  # Reraise the exception to stop the notebook

# Calculate the year-to-year growth rate for each category
funding_data['growth_rate'] = funding_data.groupby('revenue_title')['revenue_amount'].pct_change()
funding_data['growth_rate'] *= 100
funding_data.head()

In [ ]:
funding_data.dtypes

In [ ]:
funding_data['year'] = funding_data['year'].astype(int)
funding_data.dtypes

In [ ]:
funding_data.describe()

In [ ]:
funding_line = px.line(funding_data[['year', 'revenue_amount']].groupby('year')['revenue_amount'].sum().reset_index(),
                       x= 'year',
                       y= 'revenue_amount',
                       title= 'Total Revenue Trends Over Time')

funding_line.show()

In [ ]:
federal_funding = funding_data[funding_data['funding_source'] == 'federal']
federal_funding.head()

In [ ]:
state_funding = funding_data[funding_data['funding_source'] == 'state']
state_funding.head()

In [ ]:
state_funding_line = px.line(state_funding[['year', 'revenue_amount']].groupby('year')['revenue_amount'].sum().reset_index(),
                       x= 'year',
                       y= 'revenue_amount',
                       title= 'State Revenue Trends Over Time')

In [ ]:
federal_funding_line = px.line(federal_funding[['year', 'revenue_amount']].groupby('year')['revenue_amount'].sum().reset_index(),
                       x= 'year',
                       y= 'revenue_amount',
                       color_discrete_sequence=['grey'],
                       title= 'Federal Revenue Trends Over Time')

In [ ]:
TOTAL_FEDERAL_FUNDING_2011 = utils.get_year_total(federal_funding, 2011, 'revenue_amount')
TOTAL_FEDERAL_FUNDING_2015 = utils.get_year_total(federal_funding, 2015, 'revenue_amount')
PERCENT_DIFF_FEDERAL_FUNDING_2011_2015 = utils.calculate_percentage_difference(TOTAL_FEDERAL_FUNDING_2011, TOTAL_FEDERAL_FUNDING_2015)

print(f"Total Federal Funding 2011: ${TOTAL_FEDERAL_FUNDING_2011}")
print(f"Total Federal Funding 2015: ${TOTAL_FEDERAL_FUNDING_2015}")
print(f"Percent Difference from 2011 to 2015: {PERCENT_DIFF_FEDERAL_FUNDING_2011_2015:.2f}%")
print()

TOTAL_FEDERAL_FUNDING_2016 = utils.get_year_total(federal_funding, 2016, 'revenue_amount')
TOTAL_FEDERAL_FUNDING_2020 = utils.get_year_total(federal_funding, 2020, 'revenue_amount')
PERCENT_DIFF_FEDERAL_FUNDING_2016_2020 = utils.calculate_percentage_difference(TOTAL_FEDERAL_FUNDING_2016, TOTAL_FEDERAL_FUNDING_2020)

print(f"Total Federal Funding 2016: ${TOTAL_FEDERAL_FUNDING_2016}")
print(f"Total Federal Funding 2020: ${TOTAL_FEDERAL_FUNDING_2020}")
print(f"Percent Difference from 2016 to 2020: {PERCENT_DIFF_FEDERAL_FUNDING_2016_2020:.2f}%")

In [ ]:
grouped_state_revenue = state_funding.groupby('year')['revenue_amount'].sum().reset_index()
grouped_state_revenue['growth_rate'] = grouped_state_revenue['revenue_amount'].pct_change() * 100
grouped_state_revenue.head(11)

In [ ]:
TOTAL_STATE_FUNDING_2010 = utils.get_year_total(state_funding, 2010, 'revenue_amount')
TOTAL_STATE_FUNDING_2013 = utils.get_year_total(state_funding, 2013, 'revenue_amount')
TOTAL_STATE_FUNDING_2014 = utils.get_year_total(state_funding, 2014, 'revenue_amount')
TOTAL_STATE_FUNDING_2020 = utils.get_year_total(state_funding, 2020, 'revenue_amount')
PERCENT_DIFF_STATE_FUNDING_2010_2013 = utils.calculate_percentage_difference(TOTAL_STATE_FUNDING_2010, TOTAL_STATE_FUNDING_2013)
PERCENT_DIFF_STATE_FUNDING_2013_2014 = utils.calculate_percentage_difference(TOTAL_STATE_FUNDING_2013, TOTAL_STATE_FUNDING_2014)

MEAN_GROWTH_RATE_2014_2020 = utils.calculate_mean_growth_rate(grouped_state_revenue,
                                                              start_year= 2014,
                                                              end_year=2020)

PERCENT_DIFF_STATE_FUNDING_2014_2020 = utils.calculate_percentage_difference(TOTAL_STATE_FUNDING_2014, TOTAL_STATE_FUNDING_2020)

print(f"Total State Funding 2010: ${TOTAL_STATE_FUNDING_2010}")
print(f"Total State Funding 2013: ${TOTAL_STATE_FUNDING_2013}")
print(f"Total State Funding 2014: ${TOTAL_STATE_FUNDING_2014}")
print(f"Total State Funding 2020: ${TOTAL_STATE_FUNDING_2020}")
print(f"Percent Difference from 2010 to 2013: {PERCENT_DIFF_STATE_FUNDING_2010_2013:.2f}%")
print(f"Percent Difference from 2013 to 2014: {PERCENT_DIFF_STATE_FUNDING_2013_2014:.2f}%")
print(f"Mean Growth Rate from 2014 to 2020: {MEAN_GROWTH_RATE_2014_2020:.2f}%")
print(f"Percent Difference from 2014 to 2020: {PERCENT_DIFF_STATE_FUNDING_2014_2020:.2f}%")

In [ ]:
combined_funding_line = utils.create_combined_figure(federal_funding_line, 
                                                    state_funding_line,
                                                    'Funding Trends Over Time by Funding Source',
                                                    ('Federal Revenue Trends', 'State Revenue Trends'))
combined_funding_line.show()

#### Notes:

**Federal Funding**  
Between 2011 and 2015, there was a significant negative trend in federal funding, resulting in a -21.23% decrease in funding in the selected funding categories. 2016 was the start of an overall increase in funding  (slight decrese between 2017 and 2018), resulting in an overall 8.46% increase through 2020.

- Total Federal Funding 2011: $33.35B
- Total Federal Funding 2015: $26.27B
- Percent Difference from  2011 to 2015: -21.23%

- Total Federal Funding 2016: $26.85B
- Total Federal Funding 2020: $29.13B
- Percent Difference from  2016 to 2020: 8.46%

**State Funding**  
Between 2010 and 2013, there was an increase of 5.37% in the selected funding categories. There was a sudden uptick in funding between 2013 and 2014, resulting in a 7.82% increase. The trend steadies after 2014, resulting in a mean growth rate of 4.04% between 2014 and 2020, for an overall 22.26% increase.

- Total State Funding 2010: $199.24B
- Total State Funding 2013: $209.93B
- Total State Funding 2014: $226.34B
- Total State Funding 2020: $276.73B
- Percent Difference from 2010 to 2013: 5.37%
- Percent Difference from 2013 to 2014: 7.82%
- Mean Growth Rate from 2014 to 2020: 4.04%
- Percent Difference from 2014 to 2020: 22.26%

In [ ]:
expenditure_vs_funding_trends = utils.create_combined_figure(tech_voc_expenditures_line_chart1, 
                                                             funding_line, 
                                                             "National Trends in Expenditures & Funding Over Time",
                                                             ('Total Tech & Vocational Expenditures', 'Total Funding (Federal & State Sourced)'))

expenditure_vs_funding_trends.show()

#### Notes:  
While there are limitations to the level of granularity for specific funding vs spending allocations in the chosen categories for this study, we can see similar trends between expenses and funding between 2010 and 2020. Overall, there is continued growth, with the most significant growth occuring after 2014. 

<a id='visualizations_summaries'></a>
## Visualizations & Summaries

**Note:** This section consolidates all pertinent visualizations and comprehensive notes summarizing findings into a single area. The sequential arrangement aims to present a cohesive overview of the analysis, allowing for easy comprehension and detailed examination of trends, patterns, and critical insights derived from the data.

In [ ]:
tech_vocational_line_charts_combined.show()

### Notes:

Original Analysis:  

**Total Expenditures Over Time:**  
- There is a steady positive growth in Tech and Vocational expenditures from 2010 to 2014.
- In 2014, there is a sharp increase in expenditures ending in 2016.
- In 2016, expenditures appear to steady.
- Positive growth is visible throughout. 

**Expenditures by Title Over Time:**  
- After breaking the data out by Expenditure Title, we see that in 2015, two additional expenditure categories were added:
    - Tech Related Supplies & Services
    - Tech Related Equipment
- There is fairly consistent positive growth in Tech Related Equipment between 2015 and 2019.
- After 2019, there is a slight decline in this expenditure category. 
    - It should be noted that COVID-19 started spreading in late 2019, which lead to worldwide pandemic in 2020. 
- Tech Related Supplies & Services appears to have the largest impact to the growth exhibited between 2015 and 2020.
- Teacher Salaries Vocational Education remains constant.  

### Additional Analysis:  
- Difference between 2014 & 2016: $ 4,350,353,000  
- Percent difference between 2014 & 2016: 82.64%  

- Teacher Salaries Vocational Education % of Total Tech and Vocational Expenditures from 2014 to 2016: 75.78%  
    - Total: $16,308,173,000  

- Tech Related Supplies & Services percent of Total Tech and Vocational Expenditures from 2014 to 2016: 18.35%  
    - Total: $3,948,099,000  
    
- Tech Related Equipment percent of Total Tech and Vocational Expenditures from 2014 to 2016: 5.87%  
    - Total: $1,263,890,000

#### Summary:  
Between 2014 and 2016, there was an 82.64% increase in tech and vocational education related expenditures. Upon further investigation, this increase was driven by the addition of two expenditure categories, `Tech Related Supplies Services` and `Tech Related Equipment`. The main driving factor in the expenditure spike was due to `Tech Related Supplies Services`, which accounted for 18.35% of the total expenditures between 2014 and 2016. Although we see consistent growth in `Teacher Salaries Vocational Edication`, this still accounted for 75.78% of expenditures between 2014 and 2016. After 2016, the sharp spike in expenditures slows, but still maintains a positive trend. `Tech Related Supplies Services` continues to have a greater influence to the overall trend despite the spike in expenditures slowing after 2016.

#### Noteworthy Event in 2015:

The **Every Student Succeeds Act (ESSA)**, enacted in 2015, marked the transition from the **No Child Left Behind Act (NCLB)** and likely impacted educational spending trends. Both **NCLB** and **ESSA** were centered on standards-based education reform, emphasizing accountability and federal funding distribution across various educational domains. However, their primary focus wasn't explicitly geared toward technology and vocational education allocations. Despite lacking specific dedicated funding, their broader influence on education funding, accountability measures, and state discretion might have indirectly affected these areas.

#### Food for Thought: 
- There is a steady increase in `Teacher Salaries Vocational Education` over the last decade. Is this change driven by higher teacher salaries, more vocational class offerings resulting in more teachers being hired, or is it a combination of both?
- What are the rankings of expenditures by state?
- What are the rankings of individual school districts within each state that are driving these trends?
- Do student counts per school district/state have any impact on these trends?

In [ ]:
tech_vocational_cost_per_student_line_charts_combined.show()

### Notes:
The cost per student in tech and vocational education expenditures reveals significant growth trends, especially between 2014 and 2016, as observed in the line charts. This period marked a substantial 114.38% increase in costs per student, totaling an increase of $6305.10.

**Teacher Salaries - Vocational Education**
- `teacher_salaries_vocational_education` demonstrates an overall upward trend.
- The average annual growth rate from 2013 to 2019 was 3.27%.
- A significant rise of 7.35% was recorded in 2020, indicating an exceptional growth year.

**Tech-Related Equipment**
- The tech_related_equipment category experienced its highest growth between 2015 and 2016, at 68.56%.
- From 2016 to 2020, the average annual growth rate was 21.36%, showing sustained growth over the period.

**Tech-Related Supplies & Services**
- The tech_related_supplies_services category witnessed its most substantial increase between 2015 and 2016, at 146.71%.
- Although the growth rate decreased after 2016, it still maintained a high average annual growth rate of 42.20% until 2020.

In [ ]:
student_population_combined.show()

#### Notes:
- The student population between 2010 and 2015 grew by approximately 215,966 individuals, resulting in a percentage increase of about 0.45%. Despite the decline observed in 2012, the overall trend still showcases substantial growth in the student population over this five-year span.

### Funding Analysis Scope and Limitations

**Funding Titles Included:**

The analysis encompasses specific funding titles across federal and state sources. These titles were chosen based on their potential contribution to expenditure trends related to tech/vocational education. The included funding titles are as follows:

#### Federal Funding Sources:
- "21st_century_learning_centers_thru_state"
- "bilingual_education_thru_state"
- "effective_instruction_support_thru_state"
- "esser_fund"
- "geer_fund"
- "indiv_with_disabilities_thru_state"
- "math_science_teacher_quality_thru_state"
- "student_support_academic_enrich_thru_state"
- "title_I_thru_state"
- "voc_tech_education_thru_state"

#### State Funding Sources:
- "bilingual_education_state"
- "compensatory_basic_skills_programs"
- "general_formula_assistance"
- "special_education_programs"
- "vocational_education_programs"

**Analysis Overview:**

The subsequent analysis presents a generalized view of expenditure trends attributed to these specific funding categories. However, it's important to note that while these categories might provide insights into the overall spending patterns, the analysis lacks granularity. We cannot determine the exact amount allocated or spent specifically towards tech/vocational education within each category. Therefore, the trends identified serve as indicators or drivers rather than precise figures for tech/vocational education expenditure.

In [ ]:
combined_funding_line.show()

#### Notes:

**Federal Funding**  
Between 2011 and 2015, there was a significant negative trend in federal funding, resulting in a -21.23% decrease in funding in the selected funding categories. 2016 was the start of an overall increase in funding  (slight decrese between 2017 and 2018), resulting in an overall 8.46% increase through 2020.

- Total Federal Funding 2011: $33.35B
- Total Federal Funding 2015: $26.27B
- Percent Difference from  2011 to 2015: -21.23%

- Total Federal Funding 2016: $26.85B
- Total Federal Funding 2020: $29.13B
- Percent Difference from  2016 to 2020: 8.46%

**State Funding**  
Between 2010 and 2013, there was an increase of 5.37% in the selected funding categories. There was a sudden uptick in funding between 2013 and 2014, resulting in a 7.82% increase. The trend steadies after 2014, resulting in a mean growth rate of 4.04% between 2014 and 2020, for an overall 22.26% increase.

- Total State Funding 2010: $199.24B
- Total State Funding 2013: $209.93B
- Total State Funding 2014: $226.34B
- Total State Funding 2020: $276.73B
- Percent Difference from 2010 to 2013: 5.37%
- Percent Difference from 2013 to 2014: 7.82%
- Mean Growth Rate from 2014 to 2020: 4.04%
- Percent Difference from 2014 to 2020: 22.26%

In [ ]:
expenditure_vs_funding_trends.show()

#### Notes:  
While there are limitations to the level of granularity for specific funding vs spending allocations in the chosen categories for this study, we can see similar trends between expenses and funding between 2010 and 2020. Overall, there is continued growth, with the most significant growth occuring after 2014. 

#### Summary

#### Key Findings

1. **Expenditure Trends (2010-2020):**
   - Steady growth in tech and vocational education expenditures from 2010 to 2014.
   - A sharp increase in expenditures from 2014 to 2016, followed by stabilization.
   - Consistent growth throughout the decade, with the most notable spike between 2014 and 2016 (82.64% increase).

2. **Expenditures by Title (2010-2020):**
   - Introduction of new expenditure categories in 2015: Tech Related Supplies & Services, and Tech Related Equipment.
   - Significant growth in Tech Related Supplies & Services, largely influencing the expenditure trends from 2015 to 2020.
   - Teacher Salaries in Vocational Education remained constant, comprising 75.78% of total expenditures between 2014 and 2016.

3. **Impact of Legislative Changes:**
   - The enactment of the Every Student Succeeds Act (ESSA) in 2015, transitioning from No Child Left Behind Act (NCLB), might have indirectly influenced funding trends.

4. **Additional Considerations:**
   - Queries about the increase in Teacher Salaries Vocational Education, the rankings of expenditures by state and district, and the impact of student counts on trends.

5. **Cost Per Student Analysis:**
   - A 114.38% increase in cost per student between 2014 and 2016.
   - Significant yearly growth in teacher salaries and tech-related categories.

6. **Student Population Trends:**
   - A 0.45% increase in student population from 2010 to 2015, despite a temporary decline in 2012.

7. **Funding Analysis Scope and Limitations:**
   - Analysis based on specific federal and state funding sources, lacking granularity in the exact allocation towards tech/vocational education.
   - Trends serve as indicators rather than precise figures.

8. **Federal and State Funding Trends:**
   - Federal funding: A decrease of 21.23% from 2011 to 2015, followed by an 8.46% increase through 2020.
   - State funding: A steady increase from 2010 to 2020, with significant growth after 2014.

#### Conclusion
The analysis reveals a complex and dynamic landscape of tech and vocational education funding in the US from 2010 to 2020. Key highlights include the substantial growth in expenditures, especially post-2014, and the introduction of new expenditure categories. Legislative changes, like the transition to ESSA, and the broader economic and educational context, including the COVID-19 pandemic, have likely influenced these trends. The analysis, while comprehensive, points to the need for further investigation into specific allocations and the impact of different factors on these trends.